# Web Scraper + MongoDB
Daten werden von der Seite https://www.nasdaq.com/market-activity/earnings gewonnen und in der Datenbank DB_Earnings abgespeichert.

In [1]:
from pymongo import MongoClient
from selenium import webdriver
from datetime import datetime as DateTime
from datetime import date
import time

# Verbindung zur Datenbank

In [2]:
client = MongoClient("mongodb+srv://user1:@datenbankearnings.9f0rr.mongodb.net/DatenbankEarnings?retryWrites=true&w=majority")
db = client.get_database("DB_Earnings")
aktien_tabelle = db.Aktie
data_tabelle = db.Daten

## Web Scraper NASDAQ/Earnings
Dieser Abschnitt implementiert einen Web Scraper, der die Seite NASDAQ/Earnings durch geht und die Daten für jede Aktie sammelt.

In [3]:
def strtoArray(symbol_String):
    csv_array = []
    string = ""
    for i in symbol_String:
        if i == " ":
            csv_array.append(string)
            string = ""
        else:
            string = string + i
    return csv_array

In [4]:
def klick(web,tag):
    print(tag)
    if(tag < 10):
        tag_str= "0" + str(tag)
    else:
        tag_str = str(tag)
    button = web.find_element_by_xpath("//button [@data-day = '" + tag_str + "']")
    button.click()
    time.sleep(5)

In [5]:
def auslesen(filter_true, fitler_mrd,tag):
    driver = webdriver.Safari(executable_path="/usr/bin/safaridriver")
    driver.maximize_window()
    driver.get("https://www.nasdaq.com/market-activity/earnings")
    klick(driver,tag)
    content = driver.find_elements_by_class_name("market-calendar-table__row")
    symbol_key = 0  # 1 -> Pre Market; 2 -> After Hours; 3 -> Time not supplied
    daily_array = []
    j = 2 # falls Daten im voraus gelesen werden sollen
    for symbol in content:
        symbol_string = symbol.text
        symbol_string = symbol_string.replace(" ", "")
        symbol_string = symbol_string.replace("\n", " ")
        symbol_array = strtoArray(symbol_string)
        symbol_array = list(filter(None, symbol_array))
        img_src = symbol.find_element_by_class_name("market-calendar-table__icons").get_attribute("src")
        if "time-pre-market" in img_src:
            symbol_key = 1
        elif "time-after-hours" in img_src:
            symbol_key = 2
        elif "time-not-supplied" in img_src:
            symbol_key = 3
        else:
            print("Fehler - key")
        symbol_array.append(symbol_key)
        symbol_array_wert = symbol_array[2 + j].replace(",", "")
        symbol_array_wert = symbol_array_wert.replace("$", "")
        if (int(symbol_array_wert.replace("(","").replace(")","")) > (fitler_mrd * (10 ** 9)) and filter_true):
            symbol_array.append(symbol_array_wert)
            daily_array.append(symbol_array)
    driver.close()
    return daily_array

# Aktien in MongoDB Tabelle schreiben

## Create JSON

In [6]:
def create_json_aktie(aktie):
    aktie_json = {}
    aktie_json["Symbol"] = aktie[0]
    aktie_json["Name"] = aktie[1]
    aktie_json["Marktkapitalisierung"] = int(aktie[len(aktie)-1])
    return aktie_json

In [7]:
def create_json_data(aktie,tag,monat):
    data_json = {}
    data_json["Symbol"] = aktie[0]
    eps = aktie[2].replace("$","")
    eps = eps.replace("(","")
    eps = eps.replace(")","")
    data_json["EPS"] = float(eps.replace(",",""))
    data_json["Surprise"] = float(aktie[3])
    data_json["Datum"] = str(tag) + "."+ str(monat) + DateTime.now().strftime(".%y%y")
    return data_json

## Insert/Update Daten und Aktien in DB

In [8]:
def data_to_DB(aktie,tag,monat):
    data_json = create_json_data(aktie,tag,monat)
    if(not bool(data_tabelle.find_one(data_json))):
        data_tabelle.insert_one(data_json)

In [9]:
def daily_list_to_DB(daily_list,tag,monat):
    for aktie in daily_list:
        aktie_json = create_json_aktie(aktie)
        data_to_DB(aktie,tag,monat) 
        print(aktie)
        if(not bool(aktien_tabelle.find_one({"Symbol":aktie_json["Symbol"]}))):
            aktien_tabelle.insert_one(aktie_json)
            print("Insert")
        else:
            symbol = aktie_json.pop("Symbol")
            aktie_json.pop("Name")
            aktien_tabelle.update_one({"Symbol":symbol},{"$set":aktie_json})
            print("Update")
        print("_____________")

# Start

In [10]:
def start_scraping(filter_true, filter_mrd):
    tag = int(DateTime.now().strftime('%d'))
    monat = int(DateTime.now().strftime('%m'))
    for i in range(3):
        if(tag == 1):
            if(monat%2):
                tag = 31
                monat =-1
            else:
                tag = 30
                monat =-1
        else:
            tag = tag - 1
        print(tag)
        daily_list = auslesen(filter_true,filter_mrd, tag)
        daily_list_to_DB(daily_list,tag,monat)


In [11]:
start_scraping(True,10)

14
14
13
13
['LI', 'LiAutoInc.', '($0.08)', '-14.29', '$26,579,017,245', 'Sep/2020', '($0.07)', '1', 3, '26579017245']
Update
_____________
['DKNG', 'DraftKingsInc.', '($0.98)', '-55.56', '$15,602,476,477', 'Sep/2020', '($0.63)', '6', 3, '15602476477']
Update
_____________
['VIPS', 'VipshopHoldingsLimited', '$0.25', '4.17', '$14,973,107,066', 'Sep/2020', '$0.24', '1', 3, '14973107066']
Update
_____________
12
12
['DIS', 'WaltDisneyCompany(The)', '($0.20)', '70.59', '$244,893,227,224', 'Sep/2020', '($0.68)', '9', 3, '244893227224']
Update
_____________
['CSCO', 'CiscoSystems,Inc.', '$0.68', '7.9399999999999995', '$163,226,826,191', 'Oct/2020', '$0.63', '9', 3, '163226826191']
Update
_____________
['PDD', 'PinduoduoInc.', '($0.10)', '58.33', '$160,733,418,609', 'Sep/2020', '($0.24)', '3', 3, '160733418609']
Update
_____________
['AMAT', 'AppliedMaterials,Inc.', '$1.25', '6.84', '$63,747,136,927', 'Oct/2020', '$1.17', '9', 3, '63747136927']
Update
_____________
['BAM', 'BrookfieldAssetMan